<a href="https://colab.research.google.com/github/joaochenriques/IST_MCTE/blob/main/ChannelFlows/Simulation/ChannelFlow_Assignment_2023_V02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import matplotlib.pyplot as mpl
import matplotlib.ticker as plticker
import numpy as np
from dataclasses import dataclass
from scipy.optimize import minimize_scalar
import pathlib, subprocess, os

In [ ]:
def cmdcall( cmd ):
    output = subprocess.getoutput( cmd )
    print(output)

In [ ]:
if not pathlib.Path("mpl_utils.py").exists():
  cmdcall( "curl -O https://raw.githubusercontent.com/joaochenriques/ipynb_libs/main/mpl_utils.py" ) 

import mpl_utils as mut
mut.config_plots()

from matplotlib_inline.backend_inline import set_matplotlib_formats
set_matplotlib_formats('svg')

In [ ]:
try:
    from tqdm.notebook import tqdm
except ModuleNotFoundError:
    cmdcall( 'pip install tdqm' )
    from tqdm.notebook import tqdm

from IPython.display import Markdown, display
def printmd(string):
    display( Markdown(string) )

# **Setup the problem**

The blockage factor per turbine row $i$ is

$$B_i=\displaystyle \frac{\left( n_\text{T} A_\text{T}\right)_i}{S_i}$$

where $\left( n_\text{T} A_\text{T}\right)_i$ is the area of all turbines of row $i$, and $S_i$ is the cross-sectional area of the channel at section $i$. 

In [ ]:
twopi = 2*np.pi

@dataclass
class Config:
    ρw:float = 1025   # [kg/m³] salt water density 
    g:float   = 9.8   # [m/s²]  gravity aceleration 

    T:float = 12.0*3600.0 + 25.2*60.0 # [s] tide period

    L:float = 20000   # [m] channel length 
    h:float = 30      # [m] channel depth
    b:float = 1000    # [m] channel width
    a:float = 2.0     # [m] tidal amplitude

    Cd:float = 0.005  # [-] friction coefficient

    B_per_row: np.array = ( 0.01, ) # [-] Blockage factor per turbine row

    #===========================================================================
    def __init__( self ):
        # [m²] channel area     
        self.S = self.h * self.b     

        # [rad/s] tidal frequency
        self.ω = twopi / self.T       
        # Assuming a constant section channel we get
        c = self.L / self.S 
        # [-] frictionless channel volumetric flow rate 
        self.Q0 = self.g*self.a / (2*self.ω*c)      

        qr = self.S * np.sqrt(self.g*self.h)   # flow rate based on wave velocity
        f   = 2*self.Cd   # [-] friction coefficient used in the model is twice the value 
                          #     usual used in tidal (non standard model) 

        self.Fr_0 = self.Q0 / ( self.S * np.sqrt( self.g * self.h ) )

        self.λ_P = self.ρw * self.Q0**3 / ( self.S**2 )
        self.λ_E = self.λ_P / self.ω

        self.λ_T_star = ( 1.0 / self.S**2 ) * self.Q0**2 / ( self.g * self.a )
        self.λ_f_star = self.λ_T_star * ( f * self.L / self.h )

        printmd( "$\mathrm{Fr}_0 = %.3f$" % self.Fr_0 )
        printmd( "$\lambda_\mathrm{P} = %.3f$" % self.λ_P )
        printmd( "$\lambda_\mathrm{E} = %.3f$" % self.λ_E )
        printmd( "$\lambda_\mathrm{f}^* = %.3f$" % self.λ_f_star )
        printmd( "$\lambda_\mathrm{T}^* = %.3f$" % self.λ_T_star )
  
cfg = Config()

In [ ]:
def local_CT_and_CP( Fr4b, Fr1, B ): 

    # See Chapter 3 of the MCTE Lecture notes

    ζ4 = (1/2.)*Fr1**2 - 1/2.*Fr4b**2 + 1.0
    
    Fr4t = (Fr1 - Fr4b*ζ4 + np.sqrt(B**2*Fr4b**2 - 2*B*Fr1**2 + 2*B*Fr1*Fr4b \
            + B*ζ4**2 - B + Fr1**2 - 2*Fr1*Fr4b*ζ4 + Fr4b**2*ζ4**2))/B

    ζ4b =  (Fr1 - Fr4t*ζ4)/(Fr4b - Fr4t)
    ζ4t = -(Fr1 - Fr4b*ζ4)/(Fr4b - Fr4t)
    
    Fr2t = Fr4t*ζ4t/B

    C_T = (Fr4b**2 - Fr4t**2)/Fr1**2
    C_P = C_T*Fr2t/Fr1

    return C_T, C_P

def find_minus_CP( Fr4b, Fr1, B ): 
    # function created to discard the C_T when calling "local_CT_and_CP"
    C_T, C_P = local_CT_and_CP( Fr4b, Fr1, B ) 
    return -C_P # Minus C_P to allow minimization

In [ ]:
def compute_BCT_BCP( Fr_0, B, Q_star ):

  Fr1 = np.abs( Fr_0 * Q_star )

  if Fr1 < 1E-3:
    return 0.0, 0.0, 0.0 # all zeros

  # find the optimal C_P for the channel conditions
  res = minimize_scalar( find_minus_CP, args=(Fr1, B), bounds=[0,1], 
                      method='bounded', 
                      options={ 'xatol': 1e-08, 'maxiter': 500, 'disp': 1 } )
  Fr4b = res.x # optimal value

  C_T, C_P = local_CT_and_CP( Fr4b, Fr1, B )

  return B*C_T, B*C_P, Fr4b

# **Solution of the ODE**

$\displaystyle \frac{dQ^*}{dt^*}=\cos(t^*) - (\lambda_\text{f}^*+BC_\text{T} \lambda_\text{T}^*) \, Q^* \, |Q^*|$

$\displaystyle \frac{d E_\text{T}^*}{dt^*}= BC_\text{P} \, |{Q^*}^3|$

where $B$, $\lambda_\text{f}^*$ and $\lambda_\text{T}^*$ are constants, and $C_\text{T}$ and $C_\text{P}$ are computed as a function of the local Froude number.


This system can be writen as

$$\dfrac{d \mathbf{y}^*}{dt^*} = \mathbf{f}^*\!\!\left( \mathbf{y}^*, t^* \right),$$

with

$$\mathbf{y} = 
\begin{pmatrix}
Q^*\\
E_\text{T}^*
\end{pmatrix}
\tag{Eq. 1}
$$

and

$$
\tag{Eq. 2}
\mathbf{f}^* = 
\begin{pmatrix}
\cos(t^*) - (\lambda_\text{f}^*+BC_T \lambda_\text{T}^*) \, Q^* |Q^*|\\[4pt]
BC_P \, |{Q^*}^3|
\end{pmatrix}
$$

We adopt a first order solution of the type

$$\dfrac{\mathbf{y}^*(t_n^*+\Delta t^*)-\mathbf{y}^*(t_n^*)}{\Delta t^*} 
= \mathbf{f}^*\bigg( t_n^*, \mathbf{y}^*\left(t_n^*\right) \bigg)$$

resulting

$$\mathbf{y}^*_{n+1} = \mathbf{y}^*_n + \Delta t^* \, \mathbf{f}^*\!\!\left( t^*_n,
\mathbf{y}^*_n  \right)
\tag{Eq. 3}
$$

where

$$\mathbf{y}^*_{n}=\mathbf{y}^*(t_n^*)$$

$$\mathbf{y}^*_{n+1}=\mathbf{y}^*(t_n^*+\Delta t^*)$$


# Define RHS of the ODE, see Eq. (2)

In [ ]:
def f_star( ys, ts, λ_f_star, λ_T_star, Fr_0, B_per_row ):
    ( Q_star, E_star ) = ys 
    
    BC_T_rows = np.zeros( len( B_per_row ) )
    BC_P_rows = np.zeros( len( B_per_row ) )

    B_0 = np.nan
    for j, B in enumerate( B_per_row ): 
      # do not repeat the computations if B is equal to the previous iteration
      if B_0 != B:
        BC_T_j, BC_P_j, Fr4b = compute_BCT_BCP( Fr_0, B, Q_star )
        B_0 = B

      BC_T_rows[j] = BC_T_j
      BC_P_rows[j] = BC_P_j

    return np.array( 
              ( np.cos( ts ) - ( λ_f_star + np.sum(BC_T_rows) * λ_T_star ) * Q_star * np.abs( Q_star ), 
                np.sum(BC_P_rows) * np.abs( Q_star )**3 ) 
           )

# **Frictionless solution**

In [ ]:
periods = 4
ppp = 100 # points per period
num =  int(ppp*periods)

# stores time vector
ts_vec = np.linspace( 0, (2*np.pi) * periods, num )
Delta_ts = ts_vec[1] - ts_vec[0]

# vector that stores the lossless solution time series
ys_lossless_vec = np.zeros( ( num, 2 ) )

# solution of (Eq. 3) without "friction" term
for i, ts in enumerate( ts_vec[1:] ):
  ys_lossless_vec[i+1] = ys_lossless_vec[i] + \
                       Delta_ts * f_star( ys_lossless_vec[i], ts, 0, 0, 0, [0.0] )

# **Solution with channel bed friction and turbines thrust**

In [ ]:
# vector that stores the solution time series
ys_vec = np.zeros( ( num, 2 ) )

# solution of (Eq. 3) with "friction" terms
for i, ts in enumerate( ts_vec[1:] ):

  ys_vec[i+1] = ys_vec[i] + \
                    Delta_ts * f_star( ys_vec[i], ts, \
                                        cfg.λ_f_star, cfg.λ_T_star, cfg.Fr_0,\
                                        cfg.B_per_row )

E_star = ys_vec[:,1]
E_star_period = E_star[-1] - E_star[-ppp] # dimensionless energy in a tidal period
E_farm = E_star_period * cfg.λ_E          # convert to Joule
P_farm = E_farm / cfg.T * 1E-6            # Mean power converted to MW

printmd( r"$P_\text{turb}^\text{farm} = %.2f\,\text{MW}$" % P_farm )

In [ ]:
fig, (ax1, ax2) = mpl.subplots(1,2, figsize=(12, 4.5) )
fig.subplots_adjust( wspace = 0.2 )

n_rows = len( cfg.B_per_row )

Q_star_frictionless = ys_lossless_vec[:,0]
Q_star = ys_vec[:,0]
E_star = ys_vec[:,1]

# left figure
ax1.plot( ts_vec/twopi, Q_star, label="$n_\mathrm{rows}=%i$" % (n_rows) )
ax1.plot( ts_vec/twopi, Q_star_frictionless, label="frictionless" )
ax1.grid()
ax1.set_title( "$B_i = $" + str( cfg.B_per_row ) )
ax1.set_xlim( ( 0, 4 ) )
ax1.set_ylim( ( -1.1, 1.1 ) )
ax1.set_xlabel( '$t^*\!/\,(2\pi)$ [-]')
ax1.set_ylabel( '$Q^*$ [-]')
ax1.text(-0.17, 1.05, 'a)', transform=ax1.transAxes, size=16, weight='semibold')
ax1.legend( loc='upper left', fontsize=12, handlelength=2.9,labelspacing=0.25)

# right figure
ax2.plot( ts_vec/twopi, E_star )
ax2.grid()
ax2.set_title( "$B_i = %4.2f$" + str( cfg.B_per_row ) )
ax2.set_xlim( ( 0, 4 ) )
ax2.set_xlabel( '$t^*\!/\,(2\pi)$ [-]')
ax2.set_ylabel( '$E_\mathrm{T}^*$ [-]')
ax2.text(-0.17, 1.05, 'b)', transform=ax2.transAxes, size=16, weight='semibold');

mpl.savefig( 'Friction_model.pdf', bbox_inches='tight', pad_inches=0.02);

In [ ]:
def CardanoRoots( aa, bb ):
  # Cardano algorithm to solve our polynomial, see:
  # https://www.shsu.edu/kws006/professional/Concepts_files/SolvingCubics.pdf
  P = -2.0*aa
  Q = -2.0*bb
  Δ = (P/3.0)**3 + (Q/2)**2
  if Δ < 0.0: Δ = Δ + 0J
  β = ( -Q/2.0 - np.sqrt(Δ) )**(1.0/3.0)
  α = P/(3.0*β)
  ω = ( -1.0 + np.sqrt(3.0)*1J) / 2.0
  
  x1 = α - β
  x2 = (α*ω - β)*ω
  x3 = (α - β*ω)*ω

  if np.imag(x1) < 1E-15: x1 = np.real( x1 )
  if np.imag(x2) < 1E-15: x2 = np.real( x2 )
  if np.imag(x3) < 1E-15: x3 = np.real( x3 )

  # applies only for this solution 
  assert( np.imag( x1 ) == 0 )
  assert( np.imag( x2 ) == 0 )
  assert( np.imag( x3 ) == 0 )
  assert( x1 <= 0.0 )
  assert( x2 <= x3 )

  return (x2, x3)

In [ ]:
# maximum Q_star 
Q_star_max = np.max( Q_star )
 
# B for the first row
B = cfg.B_per_row[0]

# h1
h1 = cfg.h

BC_T_j, BC_P_j, Fr4b = compute_BCT_BCP( cfg.Fr_0, B, Q_star_max )

Fr1 = cfg.Fr_0 * Q_star_max

ζ4 = (1/2.)*Fr1**2 - 1/2.*Fr4b**2 + 1

C1 = Fr1 - Fr4b*ζ4
C2 = B**2*Fr4b**2 - 2*B*Fr1**2 + 2*B*Fr1*Fr4b \
        + B*ζ4**2 - B + Fr1**2 - 2*Fr1*Fr4b*ζ4 + Fr4b**2*ζ4**2

Fr4t = ( C1 + np.sqrt(C2) ) / B

ζ4 = (1/2.)*Fr1**2 - 1/2.*Fr4b**2 + 1

C1 = Fr1 - Fr4b*ζ4
C2 = B**2*Fr4b**2 - 2*B*Fr1**2 + 2*B*Fr1*Fr4b \
        + B*ζ4**2 - B + Fr1**2 - 2*Fr1*Fr4b*ζ4 + Fr4b**2*ζ4**2

Fr4t = ( C1 + np.sqrt(C2) ) / B
ζ4t = ( Fr4b*ζ4 - Fr1 ) / ( Fr4b - Fr4t )
ζ4b = ζ4 - ζ4t

mb = Fr4b*ζ4b + Fr4t*ζ4t
bb = mb**2
aa = (Fr4b**2*ζ4b + Fr4t**2*ζ4t + 1/2*ζ4**2)

ζs = CardanoRoots( aa, bb )
ζ5 = ζs[1]
Fr5 = mb / ζ5

Fr1, Fr4b, Fr4t, ζ4b, ζ4t, ζ5, Fr5

h4b = ζ4b * h1
h4t = ζ4t * h1
h5 = ζ5 * h1

sgh = np.sqrt( cfg.g * cfg. h )

U1  = Fr1 * sgh
U4b = Fr4b * sgh
U4t = Fr4t * sgh
U5  = Fr5 * sgh

U1, U4b, U4t, U5, h1, h4b, h4t, h5